ドライブをマウント（takeu...でログイン）


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 必要なパッケージのインストール
!pip install selenium retry numpy pandas requests bs4 urllib3 openai

# Seleniumのセットアップ
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from google.colab import files

# WebDriverの設定
!apt-get update
!apt-get install -y chromium-chromedriver

def web_driver():
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(options=options)
    return driver

driver = web_driver()

# その他のモジュールのインポート
import requests
from bs4 import BeautifulSoup
from retry import retry
import numpy as np
import pandas as pd
import urllib
import time
import csv
import re

# OpenAI APIの設定
import openai
openai.api_key = "sk-proj-JFVCkZdpKvV6KsN3BWu1T3BlbkFJchxeQ75YcSPvxfgvbx6i"


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.0 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy

suumoで指定エリア内の全物件情報を収集してCSV「output.csv」を出力（"max_page"に抽出量を、"url"に指定エリア検索時のURLを格納）

In [ ]:
#main

# 複数ページの情報をまとめて取得
data = []
columns_draft = []

# スクレイピングするページ数
max_page = 1
# SUUMOエリア指定して検索して出力した画面のurl(ページ数フォーマットが必要)
url='https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=060&bs=040&ta=28&sc=28222&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=25&pc=10'
# リクエストがうまく行かないパターンを回避するためのやり直し
@retry(tries=3, delay=10, backoff=2)
def load_page(url):
    html = requests.get(url)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup

# 処理時間を測りたい
start = time.time()
times = []

# ページごとの処理
for page in range(1,max_page+1):
    before = time.time()
    # ページ情報
    soup = load_page(url.format(page))
    # 物件情報リストを指定
    mother = soup.find_all(class_='cassetteitem')

    # 物件ごとの処理
    for child in mother:

        # 建物情報
        data_home = []
        # カテゴリ
        data_home.append(child.find(class_='ui-pct ui-pct--util1').text)
        # 建物名
        data_home.append(child.find(class_='cassetteitem_content-title').text)
        # 住所
        data_home.append(child.find(class_='cassetteitem_detail-col1').text)
        # 最寄り駅のアクセス
        children = child.find(class_='cassetteitem_detail-col2')
        for id,grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
            data_home.append(grandchild.text)
        # 築年数と階数
        children = child.find(class_='cassetteitem_detail-col3')
        for grandchild in children.find_all('div'):
          # if grandchild.text!='':
            data_home.append(grandchild.text)

        # 部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            data_room = []

            # 部屋情報が入っている表を探索
            for id_, grandchild in enumerate(room.find_all('td')):
                # 階
                if id_ == 2:
                    data_room.append(grandchild.text.strip())
                # 家賃と管理費
                elif id_ == 3:
                    data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
                # 敷金と礼金
                elif id_ == 4:
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
                # 間取りと面積
                elif id_ == 5:
                    data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                    data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
                # url
                elif id_ == 8:
                    get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                    abs_url = urllib.parse.urljoin(url,get_url)
                    data_room.append(abs_url)

                    # 「詳細を見る」をクリックして物件詳細情報を取得
                    soup=load_page(abs_url)
                    time.sleep(1)
                    titles=[]
                    bodies = []
                    # property_view_table-titleのリストを取得
                    titles = [title.get_text(strip=True) for title in soup.select('.property_view_table-title')]

                    # property_view_table-bodyのリストを取得

                    for body in soup.select('.property_view_table-body'):
                        if body.find('div', class_='property_view_table-read'):
                            bodies.append(body.find('div', class_='property_view_table-read').get_text(strip=True))
                        else:
                            bodies.append(body.get_text(strip=True))


                    # 指定されたテーブルだけを対象にする
                    table = soup.find('table', {'class': 'data_table table_gaiyou'})

                    # th要素と対応するtd要素を取得
                    for row in table.find_all('tr'):
                        ths = row.find_all('th', {'class': ['data_01', 'data_02']})
                        tds = row.find_all('td')

                        for th, td in zip(ths, tds):
                            titles.append(th.get_text(strip=True).replace("<br/>", ""))
                            if td.find('ul', class_='inline_list'):
                                bodies.append(td.find('ul', class_='inline_list').get_text(strip=True))
                            else:
                                bodies.append(td.get_text(strip=True))

                    # 物件情報と部屋情報をくっつける
                    titles_sample = ['タイプ', '物件名', '所在地', 'アクセス1','アクセス2', '間取り', '築年数', '階数', '階', '賃料', '管理費', '敷金', '礼金', '間取りタイプ', '面積', 'URL'] + titles
                    data_sample = data_home + data_room + bodies
                    data.append(data_sample)
                    columns_draft.append(titles_sample)
                    print(titles_sample)
                    print(data_sample)

        # 1アクセスごとに1秒休む
    time.sleep(1)

    # 進捗確認
    # このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'{page}ページ目：{running_time}秒')
    # 取得した件数
    print(f'総取得件数：{len(data)}')
    # 作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了：{complete_ratio}%')
    # 作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間：{hour}時間{minute}分{second}秒\n')

# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)

# 複数の辞書を作成
dict_list = []
for keys, values in zip(columns_draft, data):
    dictionary = dict(zip(keys, values))
    dict_list.append(dictionary)

# # 結果の表示
# print(dict_list)
# for i in range(len(dict_list)):
#   print(len(dict_list[i]),dict_list[i])

import pandas as pd
from collections import OrderedDict

# 全てのカラム名を集めて重複を削除
all_columns_draft = list(OrderedDict.fromkeys(col for sublist in columns_draft for col in sublist))

print(len(all_columns_draft))
# データフレームを初期化
df = pd.DataFrame(columns=all_columns_draft)

# data_listの辞書を元にデータフレームを作成し、all_columns_draftのカラムを設定
df = pd.DataFrame(dict_list, columns=all_columns_draft)

# 欠損値（NaN）を'-'で置き換え
df.fillna('-', inplace=True)

# CSVファイルにエンコードを指定して書き込む
csv_file_path = "/content/drive/MyDrive/Colab/garbages/outputest.csv"
df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')

print(f"CSVファイルが{csv_file_path}に出力されました。")

['タイプ', '物件名', '所在地', 'アクセス1', 'アクセス2', '間取り', '築年数', '階数', '階', '賃料', '管理費', '敷金', '礼金', '間取りタイプ', '面積', 'URL', '所在地', '駅徒歩', '間取り', '専有面積', '築年数', '階', '向き', '建物種別', '間取り詳細', '構造', '階建', '築年月', '損保', '駐車場', '入居', '取引態様', '条件', '取り扱い店舗物件コード', 'SUUMO物件コード', '総戸数', '情報更新日', '次回更新日', 'ほか初期費用', '備考']
['賃貸マンション', 'ビレッジハウスつるみ2号棟', '兵庫県養父市八鹿町伊佐', 'ＪＲ山陰本線/八鹿駅 歩38分', 'ＪＲ山陰本線/江原駅 歩68分', 'ＪＲ山陰本線/国府駅 歩121分', '築30年', '5階建', '5階', '4.45万円', '-', '-', '-', '3DK', '53.08m2', 'https://suumo.jp/chintai/jnc_000092550130/?bc=100394666846', '兵庫県養父市八鹿町伊佐', 'ＪＲ山陰本線/八鹿駅 歩38分', '3DK', '53.08m2', '築30年', '5階', '南', 'マンション', '-', '鉄筋コン', '5階/5階建', '1995年5月', '1万円2年', '敷地内2970円', '即', '貸主', '二人入居可', 'BKK-207768', '100394666846', '30戸', '2024/09/02', '次回更新日は情報更新日より１５日以内', '合計7.43万円（内訳：保険料: 10，000円 2年/退去時クリーニング費用: 64，226円(税込)）', '●短期解約違約金あり ⇒1年未満解約は賃料等の3ヵ月分、2年未満は2ヵ月分 ●法人契約の場合、賃料等の契約条件が異なる場合あり ●審査結果により敷金や連帯保証人を必要とする場合あり ●賃料に+1，000円でオプション設備追加可能 ※お部屋の仕様により設置できない場合あり 審査時必要書類：【個人】身分証・収入証明・通帳の写し【法人】会社概要（謄本等）・決算書（直近分）※必要

各物件についての情報をChatGPTに渡し、それぞれのキャッチコピーを生成


In [ ]:
# 生成したいものと目安の字数
hiroba_dict={"見出し・キャッチコピー":20,"募集動機":100, "物件の特徴・売り文句":100, "担当者の一押しコメント":100}
postscripts_dict={}
for key in hiroba_dict.keys():
    postscripts_dict[key]=[]

# 業者コメント生成、追記用辞書"postscripts_dict"に格納
for key in postscripts_dict.keys():
    print(key)
    for index, row in df.iterrows():
        info = []
        for column in df.columns:
            info.append(f'{column}は{row[column]}')
        response = openai.chat.completions.create(
          model="gpt-4o-mini-2024-07-18",
          messages=[
            {"role": "system", "content": "あなたは不動産仲介会社の、物件情報掲載担当です。"},
            {"role": "user", "content": str(hiroba_dict[key])+"文字程度でこの物件をおすすめするための魅力的な"+key+"を書いて。(URLは書かなくてよい)この部屋の"+f'{"、".join(info)}'},
            # {"role": "assistant", "content": "insert an assistance comment"},
          ]
        )

        # 生成した文言を追記用辞書に格納
        message_content = response.choices[0].message.content
        postscripts_dict[key].append(message_content)
        print(message_content)

見出し・キャッチコピー
静かな田園風景、快適3DK暮らし！
「自然に囲まれた快適3DK、即入居可能！」
「閑静な環境で心安らぐ2DK」
「ゆったり2DK！自然あふれる養父市の快適生活」
「自然豊かな八鹿で新生活！スッキリ2LDK、月5.9万円」
南向きの明るい1LDK！賃料6.2万円。充実の生活環境！
南向きの明るい1LDK♪快適な暮らしを実現！
「自然豊かな小山で、快適2LDK生活を始めよう！」
「自然と共に暮らす、快適2LDK！」
「心地よい南向き、広々79m²の2LDKタウンハウス！」
広々79㎡！自然と共に暮らす2LDKの快適空間！
「快適な2LDK、南向きの光差す暮らし」
「自然に囲まれた快適2LDK、家族におすすめ！」
自然に囲まれた静かな2LDK、理想のときめき空間！
募集動機
兵庫県養父市のビレッジハウスつるみ2号棟は、広々とした3DKの賃貸マンションで、家族にぴったりの住環境が整っています。温かみのある南向きの部屋は明るく、日当たりも良好。駐車場も完備しており、即入居可能です。初期費用もリーズナブルで、手頃な賃料で快適な暮らしを実現できます。自然豊かな環境で、穏やかな生活を始めませんか？
ビレッジハウスつるみ2号棟は、広々とした3DK（53.08m²）で家族や二人入居に最適！南向きで明るい室内、比較的静かな環境が魅力です。八鹿駅から徒歩38分、自然に囲まれた安心の住まいをご体験ください。即入居可で初期費用もお手頃です。
サニーハイネスは、緑豊かな養父市に位置する2DKの賃貸アパートです。和室と洋室があり、ゆったりとした暮らしが楽しめます。駐車場無料、JR八鹿駅までのアクセスも良好。生活費も抑えられる賃料4.5万円で新生活をスタート！
サニーハイネスは、穏やかな養父市に位置し、和洋室の2DKで使い勝手抜群。無料駐車場完備、穏やかな生活を提供します。経済的な賃料や敷金・礼金も魅力的。自然に囲まれた静かな環境で新生活を始めませんか？
兵庫県養父市の「Bloom」は、2LDKの広々としたお部屋。2024年10月上旬入居可能で、便利なクリニック前バス停から徒歩1分の好立地です。管理費2800円、賃料5.9万円でお求めやすく、駐車場も敷地内に確保。快適な新生活を始めませんか？
兵庫県養父市の「JR山陰本線 八鹿駅」近く、築10年の1LDKアパートが登場！南向き

入稿に使う関数

In [ ]:
# CSVに情報を足す関数
def update_csv_with_postscripts(csv_file_path, added_dict):

    # 付け足したい項目がcsv内にすでに存在するかどうかを確認
    for key in added_dict.keys():
        # CSVファイルを読み込む
        df = pd.read_csv(csv_file_path)
        # 1行目のカラム名をチェック
        columns = df.columns.tolist()
        print(key)
        if key not in columns:
            # 1行目最も左側の空白セルに足したいカラムを追加
            for i, col in enumerate(columns):
                if col.strip() == "":
                    columns[i] = key
                    break
            # カラム名を更新
            df.columns = columns

            # 追加するリストがCSVの行数より短い場合、繰り返しで埋める（デモ用）
            list_data = added_dict[key]
            repeat_times = (len(df) // len(list_data)) + 1  # 必要な繰り返し回数
            extended_list = (list_data * repeat_times)[:len(df)]  # リストを繰り返して必要な長さにする

            # データフレームに新しい列を追加
            df[key] = extended_list

            # 更新したデータフレームをCSVファイルとして保存
            df.to_csv(csv_file_path, index=False, encoding='utf-8-sig')
            print(f"{key}が{csv_file_path}に追加されました。")
        else:
            print(f"{key}はすでに存在します。")

# CSV内の指定したカラムと同列のセルから漢字を消す関数
def remove_kanji_from_column(csv_file_path, column_name):
    rows = []

    # CSVファイルを読み込む
    with open(csv_file_path, mode='r', encoding='utf-8') as infile:
        reader = csv.DictReader(infile)
        fieldnames = reader.fieldnames

        for row in reader:
            if column_name in row:
                value = row[column_name]
                if column_name == '階':
                    # '-' の前までの文字を残す
                    value = re.sub(r'-.*$', '', value)
                # 漢字を削除（漢字の範囲は\u4E00-\u9FFF）
                value = re.sub(r'[\u4E00-\u9FFF]+', '', value)
                value = value.replace('m2', '')  # 'm2' の削除
                row[column_name] = value
            rows.append(row)

    # CSVファイルに上書き
    with open(csv_file_path, mode='w', encoding='utf-8', newline='') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(rows)

# サイト上のラベルテキストから入力フィールドを探し、フィールドの形式に沿って入力する関数
def input_data_by_label(label_text, value):
    try:
        # ラベルテキストに対応する親要素を探す
        label_element = driver.find_element(By.XPATH, f"//*[contains(text(), '{label_text}')]")

        # ラベルテキストに対応した入力フィールドがinputかtextareaかselectかdateかそれ以外か確認し、入力する。
        try:
            input_field = label_element.find_element(By.XPATH, ".//input[1]")
            input_field.clear()
            input_field.send_keys(value)
            print(f"{label_text}に'{value}'を入力しました。")
        except Exception as e1:
            try:
                textarea_field = label_element.find_element(By.XPATH, ".//textarea[1]")
                textarea_field.clear()
                textarea_field.send_keys(value)
                # 入力した値を取得して表示
                entered_value = textarea_field.get_attribute('value')
                print(f"{label_text}に'{entered_value}'を入力しました。")
            except Exception as e2:
                try:
                    # select 要素を探す
                    select_field = label_element.find_element(By.XPATH, ".//select[1]")
                    select = Select(select_field)
                    select.select_by_visible_text(value)
                    # 選択したオプションのテキストを取得して表示
                    selected_option = select.first_selected_option.text
                    print(f"{label_text}に'{selected_option}'を選択しました。")
                except Exception as e3:
                    try:
                        # カレンダー（input type="date"）要素を探す
                        date_field = label_element.find_element(By.XPATH, ".//input[@type='date'][1]")
                        date_field.clear()
                        date_field.send_keys(value)
                        # 入力した値を取得して表示
                        entered_value = date_field.get_attribute('value')
                        print(f"{label_text}に'{entered_value}'を入力しました。")
                    except Exception as e4:
                        print(f"{label_text}のフィールドが見つかりませんでした。エラー1: {str(e1)} エラー2: {str(e2)} エラー3: {str(e3)} エラー4: {str(e4)}")
    except Exception as e:
        print(f"{label_text}の親要素が見つかりませんでした。エラー: {str(e)}")

# 指定XPathのinputを入力する関数
def input_values_from_csv_to_xpath(xpath, column_name):
    if column_name in row and row[column_name]:
        try:
            # XPathでinput要素を見つけて、CSVファイルの指定カラムの値を入力
            element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
            element.clear()
            element.send_keys(row[column_name])
            print(f"'{row[column_name]}' を入力しました。")
        except Exception as e:
            print(f"エラーが発生しました: {e}")

# 指定XPathのドロップダウンを選択する関数
def select_from_dropdown_by_csv(xpath, column_name, type_mapping):
    if column_name in row and row[column_name] not in ['', '-']:
        try:
            # XPathでドロップダウン要素を見つけて、対応する選択肢を選択
            element = wait.until(EC.presence_of_element_located((By.XPATH, xpath)))
            select = Select(element)

            if row[column_name] in type_mapping:
                select.select_by_visible_text(type_mapping[row[column_name]])
                print(f"{column_name}に'{type_mapping[row[column_name]]}'を選択しました。")
            else:
                print(f"未知の{column_name}: {row[column_name]}")
        except Exception as e:
            print(f"エラーが発生しました: {e}")

# 辞書のキーをカラムに、値のリスト内の各項目をカラムと同列のセル内に順番に格納する関数（デモ用）
def write_dict_to_csv(csv_file_path, data_dict):
    # 辞書のキーをカラム名として設定
    fieldnames = list(data_dict.keys())

    # 各カラムのリストを取得
    lists = [data_dict[key] for key in fieldnames]

    # 最大リスト長を求める
    max_length = max(len(lst) for lst in lists)

    # 各リストを同じ長さにする（ループさせる）
    extended_lists = [lst * (max_length // len(lst) + 1) for lst in lists]

    # CSVファイルに書き込む
    with open(csv_file_path, mode='w', encoding='utf-8', newline='') as outfile:
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for i in range(max_length):
            row = {key: extended_lists[idx][i] for idx, key in enumerate(fieldnames)}
            writer.writerow(row)


入稿前のCSV加工

In [ ]:
# サイトHTMLのラベルテキストとCSVカラムの対応辞書
label_to_column = {
    '見出し・キャッチコピー': '見出し・キャッチコピー',
    '募集動機': '募集動機',
    '物件の特長・売り文句': '物件の特徴・売り文句',
    '担当者の一押しコメント': '担当者の一押しコメント'
}

# サイト内ドロップダウン中の選択肢とCSV内語句の対応辞書（suumo<->不動産広場）
type_mapping = {
    '賃貸マンション': '貸マンション',
    '賃貸アパート': '貸アパート',
    '賃貸一戸建て': '貸戸建住宅',
    '賃貸テラス・タウンハウス': '貸テラスハウス'
}
structure_mapping = {
    '鉄骨鉄筋': 'ＳＲＣ（鉄骨鉄筋コンクリート造）',
    '鉄筋コン': 'ＲＣ（鉄筋コンクリート造）',
    '鉄骨': '鉄骨造（Ｓ造）',
    '軽量鉄骨': '軽量鉄骨造',
    '木造': '木造'
}
senyu_keisoku = {
    '壁芯': '壁芯',
    '内法': '内法'
}
torihiki_taiyo = {
    '貸主':'貸主',
    '代理':'代理',
    '専任':'専任',
    '仲介':'一般',
    '専属専任':'専属専任'
}
tesuryo={
    '貸主折半':'貸主折半',
    '借主折半':'借主折半',
    '当方不払':'当方不払',
    '当方全額':'当方全額',
    '当方半額':'当方半額'
}

# CSV追加分（suumoから取得できない分）
data_dict={
    '壁芯/内法':['壁芯','内法'],
    '報酬形態':['貸主折半','借主折半','当方不払','当方全額','当方半額','自由設定']
}

# 見出しなどをCSVに追加
update_csv_with_postscripts(csv_file_path, postscripts_dict)

# suumoから取得できない分をCSVに追加（デモ用）
update_csv_with_postscripts(csv_file_path, data_dict)

# 階数や賃料から漢字を削除（'3階'→'3'）
int_conversion=['階','階数','賃料','面積']
for only_int in int_conversion:
    remove_kanji_from_column(csv_file_path, only_int)

# 保存されたファイルをダウンロード
# files.download(csv_file_path)
# print(csv_file_path+"をダウンロードしました。")

見出し・キャッチコピー
見出し・キャッチコピーが/content/drive/MyDrive/Colab/garbages/outputest.csvに追加されました。
募集動機
募集動機が/content/drive/MyDrive/Colab/garbages/outputest.csvに追加されました。
物件の特徴・売り文句
物件の特徴・売り文句が/content/drive/MyDrive/Colab/garbages/outputest.csvに追加されました。
担当者の一押しコメント
担当者の一押しコメントが/content/drive/MyDrive/Colab/garbages/outputest.csvに追加されました。
壁芯/内法
壁芯/内法が/content/drive/MyDrive/Colab/garbages/outputest.csvに追加されました。
報酬形態
報酬形態が/content/drive/MyDrive/Colab/garbages/outputest.csvに追加されました。


ポータルサイトへの自動入稿
1. 不動産広場のログインページを開きログイン情報入力
2. 「新規物件登録」をクリック
3. 「賃貸居住用」をクリック
4. CSV内の各項目入力
5. 「仮登録」をクリック
6. 2.～6.を全物件情報の登録が完了するまで繰り返し行う
7. 停止


In [ ]:
# 物件情報投稿担当者名を指定
# agent_name=input("担当者名を入力してください。（例：竹内望）：")
agent_name=("竹内望") #テスト用に竹内望で指定
# CSVファイルの読み込み
df = pd.read_csv(csv_file_path)

# Beautiful Soupを使用してページを取得
url = 'https://www.fudosan-hiroba.co.jp/vendor/login'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'lxml')

# Seleniumでページを開く
driver=web_driver()
driver.get(url)

# 要素が存在するまで待つ（最大10秒待つ）
wait = WebDriverWait(driver, 10)
email_field = wait.until(EC.presence_of_element_located((By.NAME, "email")))
password_field = wait.until(EC.presence_of_element_located((By.NAME, "password")))

# メールアドレスとパスワードを入力する
email = "n.takeuchi@imas.ne.jp"  # ここにメールアドレスを入力
password = "9376"  # ここにパスワードを入力

email_field.send_keys(email)
password_field.send_keys(password)

# 入力が完了したか確認
# time.sleep(1)  # 一瞬待機してから確認

email_value = email_field.get_attribute('value')
password_value = password_field.get_attribute('value')
print(email_value, password_value)

if email_value == email and password_value == password:
    print("入力が完了しました。")
else:
    print("入力に失敗しました。")

# ログインボタンをクリックする
login_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "input.btn-login-l.shadow")))
login_button.click()

# 結果の表示
# time.sleep(1)  # ログイン後のページが読み込まれるまで待つ
current_url = driver.current_url
print("現在のURL:", current_url)

count=0
# 各物件についてフォームに入力
for index, row in df.iterrows():
    count+=1
    if count>12:
        break

    # 「新規物件登録」をクリック
    property_entry=driver.find_element(By.XPATH, '//a[text()="新規物件登録"]')
    property_entry.click()

    # 結果の表示
    # time.sleep(1)  # ログイン後のページが読み込まれるまで待つ
    current_url = driver.current_url
    print("現在のURL:", current_url)

    residence_entry=driver.find_element(By.XPATH, '//*[@id="wrap"]/div[1]/div/div[2]/table/tbody/tr[8]/th[2]/a')
    residence_entry.click()

    # 結果の表示
    # time.sleep(1)  # ログイン後のページが読み込まれるまで待つ
    current_url = driver.current_url
    print("現在のURL:", current_url)

    # 「担当者名」のドロップダウンを探す
    wait = WebDriverWait(driver, 10)
    dropdown = wait.until(EC.presence_of_element_located((By.NAME, "comment_staff_id")))

    # ドロップダウンから「竹内望」を選択
    select = Select(dropdown)
    select.select_by_visible_text(agent_name)

    # 入力が完了したか確認
    # time.sleep(1)  # 一瞬待機してから確認

    # 選択されているオプションのテキストを取得して表示
    selected_option = select.first_selected_option.text
    print(f"選択されているオプション: {selected_option}")

    # 「担当者コメント」を入力
    print(f"処理中の行 {index+1}: {row.to_dict()}")
    # 各物件について各項目入力
    for label_text, column_name in label_to_column.items():
        if column_name in row and row[column_name] not in ['', '-']:
            input_data_by_label(label_text, row[column_name])
        else:
            print(f"{column_name}はデータフレームに存在しないか、値が空もしくは'-'です。スキップします。")

    # 「タイプ」「構造」を選択
    select_from_dropdown_by_csv('//*[@id="wrap"]/div[1]/div/div[2]/form/table[1]/tbody/tr[4]/td[1]/select', 'タイプ', type_mapping)
    select_from_dropdown_by_csv('//*[@id="wrap"]/div[1]/div/div[2]/form/table[1]/tbody/tr[4]/td[2]/p/select', '構造', structure_mapping)
    select_from_dropdown_by_csv('//*[@id="wrap"]/div[1]/div/div[2]/form/div[3]/table/tbody/tr[1]/td[1]/table/tbody/tr[1]/td[1]/select', '壁芯/内法', senyu_keisoku)
    select_from_dropdown_by_csv('//*[@id="wrap"]/div[1]/div/div[2]/form/table[5]/tbody/tr[3]/td/select', '取引態様', torihiki_taiyo)
    select_from_dropdown_by_csv('//*[@id="wrap"]/div[1]/div/div[2]/form/table[6]/tbody/tr[2]/td/div/select','報酬形態',tesuryo)

    # 「階数」「階」を入力
    input_values_from_csv_to_xpath('//*[@id="wrap"]/div[1]/div/div[2]/form/table[1]/tbody/tr[4]/td[2]/div/input[1]','階数')
    input_values_from_csv_to_xpath('//*[@id="wrap"]/div[1]/div/div[2]/form/table[1]/tbody/tr[4]/td[2]/div/input[3]','階')
    input_values_from_csv_to_xpath('//*[@id="wrap"]/div[1]/div/div[2]/form/table[3]/tbody/tr[1]/td/input','賃料')
    input_values_from_csv_to_xpath('//*[@id="wrap"]/div[1]/div/div[2]/form/div[3]/table/tbody/tr[1]/td[1]/table/tbody/tr[1]/td[1]/input','面積')

    # 「仮登録」ボタンを特定してクリック
    temporary_register_button = driver.find_element(By.CSS_SELECTOR, 'input.btn-info.btn-sm[type="submit"][value="仮登録"]')
    temporary_register_button.click()

    print("仮登録ボタンがクリックされました。")

    # 現在のURLを取得
    current_url = driver.current_url
    print("現在のURL:", current_url)

    # 「この内容で仮登録してよろしいですか？」→「はい」ボタンを特定してクリック
    yes_button=driver.find_element(By.XPATH,'//*[@id="wrap"]/div[1]/div/div[2]/form/div[1]/input')
    yes_button.click()
    print("「はい」ボタンがクリックされました。")

    # 現在のURLを取得
    current_url = driver.current_url
    print("現在のURL:", current_url)

    # 結果の表示
    # time.sleep(1)  # ログイン後のページが読み込まれるまで待つ
    current_url = driver.current_url
    print("現在のURL:", current_url)

    # 結果の表示
    # time.sleep(1)  # ログイン後のページが読み込まれるまで待つ
    current_url = driver.current_url
    print("現在のURL:", current_url)

    # データ入力間隔を設定
    # time.sleep(2)  # 必要に応じて調整

# ブラウザを閉じる
driver.quit()


n.takeuchi@imas.ne.jp 9376
入力が完了しました。
現在のURL: https://www.fudosan-hiroba.co.jp/vendor/home/top
現在のURL: https://www.fudosan-hiroba.co.jp/vendor/own/entry
現在のURL: https://www.fudosan-hiroba.co.jp/vendor/own/entry/edit?category=11&is_rent=1
選択されているオプション: 竹内望
処理中の行 1: {'タイプ': '賃貸アパート', '物件名': 'ＪＲ山陰本線 八鹿駅 2階建 築14年', '所在地': '兵庫県養父市八鹿町国木', 'アクセス1': 'ＪＲ山陰本線/八鹿駅 バス15分 (バス停)クリニック前 歩1分', 'アクセス2': nan, '間取り': '2LDK', '築年数': '築14年', '階数': 2, '階': 2, '賃料': 5.9, '管理費': '2800円', '敷金': '-', '礼金': '5.9万円', '間取りタイプ': '2LDK', '面積': 57.01, 'URL': 'https://suumo.jp/chintai/jnc_000092252979/?bc=100392117674', '駅徒歩': 'ＪＲ山陰本線/八鹿駅 バス15分 (バス停)クリニック前 歩1分', '専有面積': '57.01m2', '向き': '西', '建物種別': 'アパート', '間取り詳細': '洋6 洋5.5 LDK10.00', '構造': '木造', '階建': '2階/2階建', '築年月': '2010年11月', '損保': '要', '駐車場': '敷地内3300円', '入居': "'24年10月中旬", '取引態様': '仲介', '条件': '-', '取り扱い店舗物件コード': '364-1690908', 'SUUMO物件コード': 100392117674, '総戸数': '-', '情報更新日': '2024/07/31', '次回更新日': '次回更新日は情報更新日より１５日以内', '保証会社': '保証会社利用必 契約時保証委託料：２２，０００円／月額保証委託料：賃

In [ ]:
import pandas as pd

# CSVファイルのパス
csv_file_path="/content/drive/MyDrive/Colab/garbages/outputest.csv"
# CSVファイルの読み込み
df = pd.read_csv(csv_file_path)

# カラム名の確認
print(df.columns)
print(df)


Index(['タイプ', '物件名', '所在地', 'アクセス1', 'アクセス2', '間取り', '築年数', '階数', '階', '賃料',
       '管理費', '敷金', '礼金', '間取りタイプ', '面積', 'URL', '駅徒歩', '専有面積', '向き', '建物種別',
       '間取り詳細', '構造', '階建', '築年月', '損保', '駐車場', '入居', '取引態様', '条件',
       '取り扱い店舗物件コード', 'SUUMO物件コード', '総戸数', '情報更新日', '次回更新日', '保証会社', 'ほか諸費用',
       '備考', 'ほか初期費用', '契約期間', '仲介手数料', '見出し・キャッチコピー', '募集動機', '物件の特徴・売り文句',
       '担当者の一押しコメント', '壁芯/内法', '報酬形態'],
      dtype='object')
             タイプ                  物件名           所在地  \
0         賃貸アパート  ＪＲ山陰本線 八鹿駅 2階建 築14年   兵庫県養父市八鹿町国木   
1         賃貸アパート   ＪＲ山陰本線 八鹿駅 2階建 築9年   兵庫県養父市八鹿町九鹿   
2         賃貸アパート             シャーメゾン九鹿   兵庫県養父市八鹿町九鹿   
3        賃貸マンション        ビレッジハウスつるみ1号棟   兵庫県養父市八鹿町伊佐   
4         賃貸アパート           サニーコート　Ｊｏｈ   兵庫県養父市八鹿町八鹿   
5        賃貸マンション                コーポ駅前   兵庫県養父市八鹿町八鹿   
6         賃貸アパート                小山ハイツ   兵庫県養父市八鹿町小山   
7         賃貸アパート                小山ハイツ   兵庫県養父市八鹿町小山   
8         賃貸一戸建て           グランドメゾン南II   兵庫県養父市八鹿町九鹿   
9         